# Multiple Linear Regression

In multiple linear regression we have **many** independent variables and only **one** dependent vatiable.

### 1. Assumptions of a linear regression models -->

Each LR(linear regression) model has some set of assumptions. The major of them are:
>    1. linearity,
>    2. honoscedosticity,
>    3. multivatient normality,
>    4. independance of errors, and
>    5. lack of multicollinearity.

### 2. Dummy variables -->

Dummy variables are one way to handel categorical value. The idea is to create different features derived from the different categories. Example:
> colors: {red, blue, green, red, green}

Here the three categories are {red, blue, green}. The three different columns are created what will hold boolean values. If the i<sup>th</sup> was red then only the _red_ column will have 1 in it and all other zeros. This is repeated for every training example.

This is a great way to handel categorical values, but it can lead to some problems. The major one is the _multi-collinearity_.

### 3. P value -->

Every event has some probabiity associated to it. As an example tossing a coin has 50/50 probability of giving heads and tails.

But how can the "_fairness_" of the coin be juged? How can it be juged that the coin is _fair_? This is were _hypotesis testing_ comes is.

The coin can be a fair coin or an unfair coin. Then an assumption is made about the _state_ of the coin and by tossing it the assumption is tested. It the coin is fair we expect a mix of heads and tail. But it we to see the same outcome again and again it seems to get <u>sus</u>.

That sus feeling, the point at which it feels that maybe the initial hypothesis, called the "_null hypothesis_" was incorrect is called the _**P**_ value of the hypothesis.

### 4. How to build a model? -->